In [1]:
import transformData as td
import matplotlib.pyplot as plt
import numpy as np
import pycblosc2 as cb2
import time as t

## Función que calcula los índices

In [2]:
def create_indexation(s, sb):
    dic = {}
    cont=0
    for i in range(0, s[0], sb[0]):
        for j in range(0, s[1], sb[1]):
            for k in range(0, s[2], sb[2]):
                
                K = k + j*s[2] + i*s[1]*s[2]
                dic[K]=cont
                cont+=1
    return dic

In [3]:
def obtain_index(x, y, z, dic, s, sb):

    ind = []
    
    for i in range(x[0]//32*32, x[1], sb[0]):
        for j in range(y[0]//32*32, y[1], sb[1]):
            for k in range(z[0]//32*32, z[1], sb[2]):
                
                K = k + j*s[2] + i*s[1]*s[2]
                
                ind.append(((i, j, k), dic[K]))
                
    return ind

## Definición del conjunto de datos

In [4]:
SHAPE = [512, 512, 512]

SIZE = np.prod(SHAPE)

src = np.arange(SIZE, dtype=np.int32).reshape(SHAPE)

# TRANSFORMACIÓN

start = t.perf_counter()

SUBSHAPE = [32, 32, 32]

srct = td.tData(src, SUBSHAPE, inverse=False).flatten()

dic = create_indexation(SHAPE, SUBSHAPE)

end = t.perf_counter()

ttran_tra = end - start

## Compresión y decompresión de los datos

In [5]:
KB = 1024
MB = 1024 * KB
GB = 1024 * MB

BLOSC_MAX_FILTERS = 5
BLOSC_BLOCKSIZE = 16 * KB
BLOSC_TYPESIZE = 4
BLOSC_CODE = 5

cparams = cb2.blosc2_create_cparams(compcode=BLOSC_CODE, clevel=5, use_dict=0, typesize=BLOSC_TYPESIZE,
                                        nthreads=4, blocksize= BLOSC_BLOCKSIZE, schunk=None,
                                        filters=[0, 0, 0, 0, 1],
                                        filters_meta=[0, 0, 0, 0, 0])

dparams = cb2.blosc2_create_dparams(nthreads=4, schunk=None)

### Datos sin transformación

In [104]:
start = t.perf_counter()

BSIZE = SIZE * src.dtype.itemsize

schunk = cb2.blosc2_new_schunk(cparams, dparams)
nchunks = cb2.blosc2_append_buffer(schunk, BSIZE, src)

end = t.perf_counter()

torig_com = end - start

In [105]:
start = t.perf_counter()

dest = np.zeros(SIZE, dtype=np.int32).reshape(SHAPE)
cb2.blosc2_decompress_chunk(schunk, 0, dest, BSIZE)

res = dest[:,130,12]

cb2.blosc2_free_schunk(schunk)

end = t.perf_counter()

torig_dec = end - start

### Datos transformados

In [106]:
start = t.perf_counter()

schunk = cb2.blosc2_new_schunk(cparams, dparams)

AUX_SIZE = np.prod(SUBSHAPE)
AUX_BSIZE = AUX_SIZE * srct.dtype.itemsize

for i in range(SHAPE[0]//SUBSHAPE[0] * SHAPE[1]//SUBSHAPE[1] * SHAPE[1]//SUBSHAPE[1]):
    aux = srct[i * AUX_SIZE:(i+1) * AUX_SIZE]
    nchunks = cb2.blosc2_append_buffer(schunk, AUX_BSIZE, aux)
end = t.perf_counter()

ttran_com = end - start

In [107]:
start = t.perf_counter()

SUBPL = (SHAPE[0], SUBSHAPE[1], SHAPE[2])

ind = obtain_index((0,SHAPE[0]), (130,131), (12,13), dic, SHAPE, SUBSHAPE)

dest = np.zeros(np.prod(SUBPL), dtype=np.int32).reshape(SUBPL)

AUX_SIZE = np.prod(SUBSHAPE)
AUX_BSIZE = AUX_SIZE * srct.dtype.itemsize

aux = np.zeros(AUX_SIZE, dtype=np.int32).reshape(SUBSHAPE)

for (i, j, k), n in ind:
    cb2.blosc2_decompress_chunk(schunk, n, aux, AUX_BSIZE)
    dest[i:i+SUBSHAPE[0], 0:SUBSHAPE[1], k:k+SUBSHAPE[2]] = aux
    
res2 = dest[:, 130%SUBSHAPE[1], 12%SUBSHAPE[2]]

end = t.perf_counter()

ttran_dec = end - start

## Resultados

In [108]:
print('La compresión de los datos transformados es {:.3f}x más rápida que la original.'.format(torig_com/(ttran_com + ttran_tra)))

La compresión de los datos transformados es 0.007x más rápida que la original.


In [109]:
print('La descompresión de los datos transformados es {:.3f}x más rápida que la original.'.format(torig_dec/ttran_dec))

La descompresión de los datos transformados es 38.292x más rápida que la original.


In [110]:
np.testing.assert_array_equal(res, res2)

In [111]:
print(torig_dec)

0.4395952009999746


In [112]:
print(ttran_dec)

0.01147996100007731
